In [14]:
import streamlit as st
import sys
import pandas as pd

dataset = st.file_uploader("dataset.csv", type = ['csv'])
if dataset is not None:
    df = pd.read_csv(dataset)
    st.write('## Data set')
    st.dataframe(df,3000,500)

In [15]:
import pandas as pd
train = pd.read_csv('dataset.csv')
print("traing set:"%train.columns , train.shape , len(train))

traing set: (31962, 3) 31962


In [17]:
test = pd.read_csv('test.csv')
print("test set: "% test.columns,test.shape,len(test))



test set:  (17197, 2) 17197


In [19]:
train

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."


In [22]:
import re
def clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem:re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
    return df

In [23]:
test_clean=clean_text(test, "tweet")
train_clean=clean_text(train,"tweet")

In [24]:
from sklearn.utils import resample
train_majority = train_clean[train_clean.label==0]
train_minority = train_clean[train_clean.label==1]

train_minority_upsampled = resample(train_minority,replace=True,n_samples=len(train_majority),random_state=123)
train_upsampled = pd.concat([train_minority_upsampled,train_majority])
train_upsampled['label'].value_counts()

1    29720
0    29720
Name: label, dtype: int64

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from sklearn.linear_model import SGDClassifier
pipeline_sgd=Pipeline([('vect',CountVectorizer()),
                      ('tfidf',TfidfTransformer()),
                      ('nb',SGDClassifier()),])


In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_upsampled['tweet'],train_upsampled['label'],random_state=0)

In [48]:
from sklearn.pipeline import make_pipeline
model = pipeline_sgd.fit(X_train,y_train)

In [49]:
y_predict = model.predict(X_test)

In [50]:
from sklearn.metrics import f1_score
f1_score(y_test, y_predict)

0.9696646443096207